# Behavioural Planning for Autonomous Highway Driving

We plan a trajectory using the _Optimistic Planning for Deterministic systems_ ([OPD](https://hal.inria.fr/hal-00830182)) algorithm.


In [ ]:
#@title Imports for env, agent, and visualisation.
# Environment
!pip install highway-env
import gymnasium as gym
import highway_env

# Agent
!pip install git+https://github.com/eleurent/rl-agents#egg=rl-agents
from rl_agents.agents.common.factory import agent_factory

# Visualisation
import sys
from tqdm.notebook import trange
!pip install moviepy -U
!pip install imageio_ffmpeg
!pip install pyvirtualdisplay
!apt-get install -y xvfb python-opengl ffmpeg
!git clone https://github.com/eleurent/highway-env.git
sys.path.insert(0, './highway-env/scripts/')
from utils import record_videos, show_videos


In [1]:
import gymnasium as gym
import numpy as np 
from datetime import datetime
import csv
np.random.seed(1)
output = datetime.now().strftime("%m_%d_%H:%M")
output = output + ".npy"
env = gym.make('highway-v0', render_mode='rgb_array')
env.configure({
    "manual_control": True,
    "observation": {
        "type": "Kinematics",
        "vehicles_count": 3,
        "features": ["car_id","presence", "x", "y", "vx", "vy"],
        "features_range": {
            "x": [-100, 100],
            "y": [-100, 100],
            "vx": [-20, 50],
            "vy": [-20, 50]
        },
        "absolute": False,
        "order": "sorted"
    }
})
print(env.config)
obs, info = env.reset(seed = 1) #collect a single episode (replay for later)
print("Obs:", obs)
done = truncated = False
data = []
reward = 0
while not (done or truncated):
   action = env.action_space.sample() 
   data.append({'action': action, 'reward': reward, 'obs': obs, 'info': info, 'done':done})
   print("Action:", action)
   obs = env.render()
   obs, reward, done, truncated, info = env.step(action)
   print("obs", obs)
 
np.save(output, data)
print("Saved to:", output)
env.close()

THIS IS THE RESET!
{'observation': {'type': 'Kinematics', 'vehicles_count': 3, 'features': ['car_id', 'presence', 'x', 'y', 'vx', 'vy'], 'features_range': {'x': [-100, 100], 'y': [-100, 100], 'vx': [-20, 50], 'vy': [-20, 50]}, 'absolute': False, 'order': 'sorted'}, 'action': {'type': 'DiscreteMetaAction'}, 'simulation_frequency': 15, 'policy_frequency': 1, 'other_vehicles_type': 'highway_env.vehicle.behavior.IDMVehicle', 'screen_width': 600, 'screen_height': 150, 'centering_position': [0.3, 0.5], 'scaling': 5.5, 'show_trajectories': False, 'render_agent': True, 'offscreen_rendering': False, 'manual_control': True, 'real_time_rendering': False, 'lanes_count': 6, 'vehicles_count': 0, 'controlled_vehicles': 1, 'initial_lane_id': None, 'duration': 60, 'ego_spacing': 2, 'vehicles_density': 1, 'collision_reward': -1, 'right_lane_reward': 0.1, 'high_speed_reward': 0.4, 'lane_change_reward': 0, 'reward_speed_range': [20, 30], 'normalize_reward': True, 'offroad_terminal': False}
THIS IS THE RES

/Users/malhardamle/Desktop/HighwayEnv-malhar/.venv/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:42: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (2, 6)
  logger.warn(


obs [[ 0.          1.          1.          0.16        0.2857143  -0.42857143]
 [ 0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.        ]]
Action: 3
obs [[ 0.          1.          1.          0.16        0.2857143  -0.42857143]
 [ 0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.        ]]
Action: 0
obs [[ 0.          1.          1.          0.16        0.2857143  -0.42857143]
 [ 0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.        ]]
Action: 2
obs [[ 0.          1.          1.          0.16        0.2857143  -0.42857143]
 [ 0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.        ]]
Action: 4
obs [[ 0.          1.          1.       

In [ ]:
!pip show gymnasium